Aqui temos a instalação das bibliotecas necessárias

In [1]:
!pip install crewai crewai_tools langchain_community
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 12.0 MB/s eta

Importando as classes Agent, Task e Crew, as quais são respectivamente responsáveis por nos permitir criar nossos agentes, criar as tarefas que nossos agentes vão desempenhar e por fim realizar a orquestração. LLM foi inserido pois não estou conseguindo usar meus créditos da OpenAI.

In [2]:
from crewai import Agent, Task, Crew, LLM

LLM que vai se conectar aos nosso agente e credenciais, pelo fato de não conseguir usar a da OpenAI vou usar minhas credenciais do gemini da google e devido a isso vou inserir, o parâmetro llm em todos os agentes para que eles usem o modelo gemini.

In [3]:
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')
llm = LLM(model='gemini/gemini-1.5-flash',api_key=os.environ["GEMINI_API_KEY"])

Abaixo temos nosso primeiro agente o pesquisador_mercado_financeiro, o qual conta com a classe Agent definida anteriormente, a qual conta com os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é pesquisar sobre 3 tópicos que podem ser manipulados pelo usuário relacionados a economia e uma história de fundo que gera contexto para o agente. Além disso temos os parâmetros allow_delegation e verbose, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [4]:
pesquisador_mercado_financeiro = Agent(
    role="Pesquisador de Notícias do Mercado Financeiro",
    goal="Procurar notícias associadas a {palavra_chave_1}, {palavra_chave_2} e {palavra_chave_3}",
    backstory="Você é um economista-chefe da B3,"
              "que busca notícias relacionadas aos seguintes tópicos: {palavra_chave_1}, {palavra_chave_2} e {palavra_chave_3}."
              "Essas notíciias são pesquisadas dos principais portais financeiros."
              "Você coleta para cada notícia pesquisada {coluna_1}, {coluna_2}, {coluna_3} e {coluna_4}"
              "para que seja produzido uma tabela contendo as principais informações de cada notícia,"
              "a qual vai ser usado para alimentar um banco de dados."
              "Seu trabalho é procurar essas notícias.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

Abaixo temos nosso segundo agente o analista_derivativos, o qual é bastante similar ao primeiro pois conta com a classe Agent definida, a qual possui os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é pegar as notícias pesquisadas pelo primeiro agente e com base nisso procura ativos do mercado de derivativos já o backstory se trata de uma história de fundo que gera contexto para o agente detalhando mais como ele deve agir e se portar. Além disso temos os parâmetros allow_delegation e verbose assim como no agente anterior, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [5]:
analista_derivativos = Agent(
    role="Analista de Derivativos",
    goal="Identificar oportunidades no mercado de derivativos com base em notícias financeiras.",
    backstory=(
        "Um especialista em mercados financeiros com vasta experiência em análise "
        "de derivativos como futuros, opções e swaps. Seu trabalho é identificar padrões, "
        "avaliar riscos e recomendar os melhores ativos baseados em eventos de mercado."
    ),
    verbose=True,
    llm=llm
)

Abaixo temos nosso terceiro agente o economista_brasileiro, o qual é bastante similar aos demais agentes, pois conta com a classe Agent definida, a qual possui os parâmetros role, goal e backstory, os quais são responsáveis respectivamente por dizer qual a função que o agente ocupa em nossas tarefas, o seu objetivo geral ao realizar as suas tarefas é pegar os ativos que o segundo agente selecionou com base nas notícias que recebeu pelo primeiro agente e com base produz um relatório relacionando isso com a economia brasileira já o backstory se trata de uma história de fundo que gera contexto para o agente detalhando mais como ele deve agir e se portar. Além disso temos os parâmetros allow_delegation e verbose assim como no agente anterior, os quais são responsáveis respectivamente por permitir que o agente atribua tarefa para outro, que no caso do exemplo não é possível pois está como False e verbose que mostra como são os pensamentos do agente.

In [6]:
economista_brasileiro = Agent(
    role="Especialista em Economia Brasileira",
    goal="Analisar os principais eventos econômicos brasileiros e seu impacto nos ativos financeiros, fornecendo insights detalhados para investidores.",
    backstory=(
        "Sou um economista experiente, especializado no mercado financeiro brasileiro. "
        "Com anos de análise macroeconômica e conhecimento profundo sobre a política monetária, fiscal e cambial do Brasil"
        "eu traduzo dados e eventos econômicos em insights acionáveis para investidores e empresas."
    ),
    verbose=True,
    memory=True,
    llm=llm
)

Abaixo temos a a variável coletar_noticias_task, a qual é uma das tasks que nossos agentes podem realizar, dessa forma temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, sendo eles: buscar notícias relacionadas a 3 tópicos e usar eles como  palvras chaves na pesquisa onde produzimos uma tabela com 4 variáveis de escolha, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual deve ser essa tabela onde há cada notícia com representando uma linha e cada variável uma coluna, uma tabela, e o parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o planner criadfo anteriormente.

In [7]:
coletar_noticias_task = Task(
    description=(
        "Você deve pesquisar notícias financeiras nos principais portais especializados, "
        "buscando informações relacionadas aos seguintes tópicos: "
        "**{palavra_chave_1}, {palavra_chave_2} e {palavra_chave_3}**.\n\n"
        "Para cada notícia encontrada, extraia e registre as seguintes informações:\n"
        " **{coluna_1}**\n"
        " **{coluna_2}**\n"
        " **{coluna_3}**\n"
        " **{coluna_4}**\n\n"
        "**Critérios importantes:**\n"
        "- Priorize fontes **confiáveis** (ex: Valor Econômico, Bloomberg, Reuters, etc.).\n"
        "- Certifique-se de que as notícias sejam **recentes** (últimos 7 dias, se possível).\n"
        "- Se houver notícias duplicadas, mantenha apenas a mais completa e relevante.\n\n"
        "**Objetivo:**\n"
        "Gerar uma tabela estruturada contendo todas as notícias coletadas, garantindo que os dados "
        "possam ser facilmente utilizados para alimentar um banco de dados ou relatório financeiro."
    ),
    expected_output=(
        "Uma tabela bem estruturada com informações organizadas, incluindo colunas "
        "**{coluna_1}, {coluna_2}, {coluna_3}, {coluna_4}**, contendo apenas notícias "
        "relevantes e confiáveis."
    ),
    agent=pesquisador_mercado_financeiro
)

Abaixo temos a a variável tarefa_analise_derivativos, a qual é uma das tasks que nossos agentes podem realizar e assim como a anterior temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, e output esperado é uma lista com 3 ativos relacionados as notícias que ele consumiu conforme coloca a task, já parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o analista_derivativos criado anteriormente.

In [8]:
tarefa_analise_derivativos = Task(
    description=(
        "Analise as notícias financeiras mais recentes para identificar tendências e oportunidades "
        "com base nas notícias na tabela de pesquisa do agente pesquisador_mercado_financeiro, escolha 3 ativos "
        "no mercado de derivativos. Baseado nas informações, selecione pelo menos três ativos derivativos "
        "(como contratos futuros, opções ou swaps) e justifique sua escolha considerando fatores como volatilidade, "
        "indicadores econômicos e notícias relevantes."
    ),
    expected_output="Uma lista de 3 ativos derivativos recomendados, com uma breve análise justificando cada escolha.",
    agent=analista_derivativos
)

E por fim abaixo temos a variável analise_economica_task, a qual é uma das tasks que nossos agentes podem realizar e assim como a anterior temos como parâmetros, a descrição da task, onde são inseridos uma série de passos e recomendações que devem ser seguidas na realização da task, onde a idéia central é realizar uma análise econômica detalhada relacionando o contexto econômico brasileiro, o expect_output, que se trata de como deve ser a resposta do agente a tarefa, a qual pode ser um relatório, já parâmetro agent, que atribui o agente responsável pela tarefa, sendo no caso do exemplo abaixo o economista_brasileiro criado anteriormente.

In [9]:
analise_economica_task = Task(
    description=(
        "Realize uma análise detalhada dos principais eventos econômicos brasileiros recentes com base nos ativos selecionados  na lista do"
        "agente analista_derivativos e com base nisso você deve realizar uma análise relacionando com a economia brasileira e os eventos passados, para justificar essas escolhas,"
        "incluindo também as decisões do Banco Central, políticas fiscais, variação do PIB e mudanças regulatórias. "
        "Explique no relatório como esses fatores influenciaram ativos financeiros como ações, títulos públicos e câmbio. "
        "Forneça uma conclusão clara e recomendações estratégicas."
    ),
    expected_output=(
        "Um relatório em formato de artigo com pelo menos 500 palavras, abordando: "
        "- Os principais eventos econômicos recentes no Brasil e mundo "
        "- O impacto dos eventos passados no valor desses ativos. "
        "- Uma conclusão com insights para investidores."
    ),
    agent=economista_brasileiro
)

Abaixo temos a variável crew, a qual recebe a classe Crew, onde temos os seguintes parâmetros agents, tasks e verbose, os quais respectivamente se tratam de uma lista de agentes os quais vão executar as tarefas, a lista de tarefas a serem executadas por nossos agentes e os pensamentos de nossos agentes ao executar as tarefas. Além vale destacar que nesse caso as tarefas são executadas sequencialmente, isto é plan -> write -> edit. Aqui diferente do tutorial a verbose teve que ser ajustado para true, devido as versões mais recentes da biblioteca.

In [10]:
crew = Crew(
    agents=[pesquisador_mercado_financeiro, analista_derivativos, economista_brasileiro],
    tasks=[coletar_noticias_task, tarefa_analise_derivativos, analise_economica_task],
    verbose=True
)

Abaixo temos a variável results a qual recebe o metódo kickoff da variável crew, onde temos os inputs que se tratam de valores interpolados que podemos substituir nos agentes, ou seja, para todas as vezes que apareceram as palvras chaves que são macroeconomia brasileira, economia brasileira e mercado brasileiro e responde as informações em colunas determinadas, as quais são dados, autor, resumo e fontes e ao executar podemos observar a execução de cada tarefa uma por uma devido ao fato de verbose estar ativa, como declarado anteriormente.


In [11]:
result = crew.kickoff(inputs={
    "palavra_chave_1": "macroeconomia brasileira",
    "palavra_chave_2": "economia brasileira",
    "palavra_chave_3": "mercado brasileiro",
    "coluna_1": "dados",
    "coluna_2": "autor",
    "coluna_3": "resumo",
    "coluna_4": "fontes"
})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7804e660-dd9b-4916-a82b-657d710ae200                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1dfcb866-4e9a-43f1-a7f8-1a8c983c95f6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1dfcb866-4e9a-43f1-a7f8-1a8c983c95f6
       Status: Executing Task...
    └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
            Status: In Progress

# Agent: Pesquisador de Notícias do Mercado Financeiro
## Task: Você deve pesquisar notícias financeiras nos principais portais especializados, buscando informações relacionadas aos seguintes tópicos: **macroeconomia brasileira, economia brasileira e mercado brasileiro**.

Para cada notícia encontrada, extraia e registre as seguintes informações:
 **dados**
 **autor**
 **resumo**
 **fontes**

**Critérios importantes:**
- Priorize fontes **confiáveis** (ex: Valor Econômico, Bloomberg, Reuters, etc.).
- Certifique-se de que as notícias sejam **recentes** (últimos 7 dias, se possível).
- Se houver notícias duplicadas, mantenha apenas a mais completa e relevante.

**Objetivo:**
Gerar uma tabela estruturada contendo todas as notícias coletadas, garantindo que os dados possam ser facilmente utilizados para alimentar um banco de dados ou relatório financeiro.


🚀 Crew: crew
└── 📋 Task: 1dfcb866-4e9a-43f1-a7f8-1a8c983c95f6
       Status: Executing Task...
    └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Especialista em Economia Brasileira
## Final Answer: 
## Análise dos Principais Eventos Econômicos Recentes no Brasil e seu Impacto nos Ativos Financeiros

Este relatório analisa os principais eventos econômicos recentes no Brasil, com base em dados hipotéticos que simulam notícias financeiras, e seu impacto em ativos financeiros selecionados: Contratos Futuros sobre Índice Ibovespa (WIN), Opções de Venda (Put) sobre Dólar Americano (USD) e Swaps de Taxa de Juros.  A análise considera o contexto histórico da economia brasileira e as decisões de política monetária e fiscal.  **É crucial lembrar que os dados utilizados são fictícios e servem apenas para ilustrar a metodologia de análise.**  Em um cenário real, uma análise completa necessitaria de dados atualizados e uma modelagem mais robusta.

**Principais Eventos Econômicos (Dados Hipotéticos):**

* **27/10/2024:** Inflação brasileira fecha outubro com alta de 0,5%, acima das expectativas do mercado, impulsionada pelo setor 

🚀 Crew: crew
├── 📋 Task: 1dfcb866-4e9a-43f1-a7f8-1a8c983c95f6
│      Assigned to: Pesquisador de Notícias do Mercado Financeiro
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
│           Status: ✅ Completed
├── 📋 Task: f422dc2e-96f2-4846-a90c-976e57613d6c
│      Assigned to: Analista de Derivativos
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista de Derivativos
│           Status: ✅ Completed
└── 📋 Task: 77e53d6f-f8fa-471a-aaa2-c67d764669ae
       Status: Executing Task...
    └── 🤖 Agent: Especialista em Economia Brasileira
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 1dfcb866-4e9a-43f1-a7f8-1a8c983c95f6
│      Assigned to: Pesquisador de Notícias do Mercado Financeiro
│      Status: ✅ Completed
│   └── 🤖 Agent: Pesquisador de Notícias do Mercado Financeiro
│           Status: ✅ Completed
├── 📋 Task: f422dc2e-96f2-4846-a90c-976e57613d6c
│      Assigned to: Analista de Derivativos
│      Status: ✅ Completed
│   └── 🤖 Agent: Analista de Derivativos
│           Status: ✅ Completed
└── 📋 Task: 77e53d6f-f8fa-471a-aaa2-c67d764669ae
       Assigned to: Especialista em Economia Brasileira
       Status: ✅ Completed
    └── 🤖 Agent: Especialista em Economia Brasileira
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 77e53d6f-f8fa-471a-aaa2-c67d764669ae                                                                     │
│  Agent: Especialista em Economia Brasileira                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7804e660-dd9b-4916-a82b-657d710ae200                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

E por fim temos a exibição de tudo organizado com nosso relatório relacionando as notícias com a economia brasileira e com os ativos.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

## Análise dos Principais Eventos Econômicos Recentes no Brasil e seu Impacto nos Ativos Financeiros

Este relatório analisa os principais eventos econômicos recentes no Brasil, com base em dados hipotéticos que simulam notícias financeiras, e seu impacto em ativos financeiros selecionados: Contratos Futuros sobre Índice Ibovespa (WIN), Opções de Venda (Put) sobre Dólar Americano (USD) e Swaps de Taxa de Juros.  A análise considera o contexto histórico da economia brasileira e as decisões de política monetária e fiscal.  **É crucial lembrar que os dados utilizados são fictícios e servem apenas para ilustrar a metodologia de análise.**  Em um cenário real, uma análise completa necessitaria de dados atualizados e uma modelagem mais robusta.

**Principais Eventos Econômicos (Dados Hipotéticos):**

* **27/10/2024:** Inflação brasileira fecha outubro com alta de 0,5%, acima das expectativas do mercado, impulsionada pelo setor de alimentos. (Valor Econômico - Simulado)
* **26/10/2024:** Banco Central reduz taxa Selic para 13,25%, em resposta à desaceleração da inflação. Analistas acreditam que novas reduções podem ocorrer nos próximos meses. (Bloomberg - Simulado)
* **25/10/2024:** Ibovespa fecha em alta de 1,2%, impulsionado por resultados positivos de empresas do setor de tecnologia. Otimismo com a economia brasileira contribui para o desempenho. (Reuters - Simulado)
* **24/10/2024:** Governo anuncia pacote de medidas para estimular o crescimento econômico, incluindo investimentos em infraestrutura e desburocratização. (Estadão - Simulado)


**Impacto nos Ativos Financeiros:**

**1. Contratos Futuros sobre Índice Ibovespa (WIN):** A alta do Ibovespa em 1,2% (25/10) e o anúncio do pacote de estímulo governamental (24/10) apontam para um cenário positivo para o mercado acionário brasileiro a curto prazo.  Contratos futuros sobre o Ibovespa permitem aproveitar essa tendência de alta.  No entanto, a inflação acima do esperado (27/10) introduz um elemento de incerteza, podendo afetar o crescimento econômico futuro e a performance das empresas listadas.  A volatilidade inerente ao mercado brasileiro, influenciada por fatores políticos e econômicos, aumenta o risco associado a essa estratégia.  A escolha do vencimento do contrato é crucial: contratos de curto prazo permitem maior alavancagem da tendência atual, mas com maior exposição à volatilidade; contratos de longo prazo oferecem uma visão mais estratégica, mas com risco aumentado caso a tendência de alta se inverta.

**2. Opções de Venda (Put) sobre Dólar Americano (USD):** A redução da Selic (26/10) sinaliza uma política monetária mais expansionista, o que, tradicionalmente, tende a depreciar a moeda local.  Opções de venda (put) sobre o dólar permitem lucrar com uma possível depreciação do real.  A escolha do *strike price* é crucial, dependendo da avaliação do risco e da expectativa de movimento cambial. A inflação mais alta que o esperado (27/10) poderia contrabalançar a pressão de baixa sobre o real, uma vez que inflação mais alta costuma atrair capitais externos em busca de maiores retornos, valorizando a moeda. Portanto, uma análise cuidadosa das projeções de inflação é fundamental para definir o *strike price* ideal.

**3. Swaps de Taxa de Juros:** A redução da Selic (26/10) e a expectativa de novas reduções criam um cenário favorável para swaps que lucram com a queda das taxas de juros.  Entretanto, a inflação acima das expectativas (27/10) introduz um risco significativo.  Se a inflação persistir em níveis elevados, o Banco Central poderá reverter sua política monetária, aumentando a Selic e prejudicando posições em swaps que apostam na queda das taxas.  A análise da curva de juros e das projeções de inflação é fundamental para determinar a estratégia ideal, incluindo a escolha do prazo do swap e do spread.

**Eventos Passados e Contexto Econômico:**

Eventos passados, como crises econômicas, políticas fiscais expansionistas ou restritivas, e choques externos, influenciam a percepção de risco dos investidores e, consequentemente, o preço dos ativos. A análise do cenário macroeconômico brasileiro, incluindo o crescimento do PIB, a inflação, a dívida pública e a taxa de câmbio, é crucial para entender o impacto dos eventos recentes.  A estabilidade política também é um fator determinante para a confiança dos investidores.

**Conclusão e Recomendações:**

Os dados hipotéticos sugerem um cenário misto para a economia brasileira.  O crescimento econômico parece estar se recuperando, impulsionado por estímulos governamentais e resultados positivos de alguns setores.  Por outro lado, a inflação persiste acima das expectativas, criando incerteza.  As decisões do Banco Central em reduzir a Selic refletem o desejo de equilibrar o crescimento econômico com o controle da inflação.

Para investidores, recomenda-se cautela e diversificação.  A alocação de recursos em ativos como os contratos futuros sobre o Ibovespa pode ser vantajosa no curto prazo, mas a volatilidade do mercado requer uma gestão de risco eficaz.  Opções de venda sobre o dólar e swaps de taxa de juros permitem estratégias mais defensivas, mas dependem de uma correta avaliação dos riscos de inflação e das expectativas sobre a política monetária.  Uma análise completa, considerando um horizonte de tempo mais amplo e dados reais atualizados, é fundamental para a tomada de decisão de investimento.  Este relatório se baseia em dados hipotéticos e não constitui recomendação de investimento.